### В этом ноутбуке подробнее познакомимся с использованием `Agents` в `LangChain`.

Архитектура агента `LangChain` выглядит следующим образом:
* **Прием ввода**: агент получает вводимые пользователем данные на естественном языке.
* **Обработка с помощью LLM**: агент использует LLM для обработки входных данных и формулирования плана действий.
* **Выполнение плана**: агент выполняет разработанный план действий, который может включать взаимодействие с другими инструментами или службами.
* **Доставка выходных данных**: впоследствии агент доставляет выходные данные выполненного плана обратно пользователю.

Существует 2 типа агентов:
* `Toolkits` агенты (агент - ящик инструментов) - заточенные под выполнение конкретных задач или для работы с конкретным API.

* `Generic` агенты (общего назначения) - универсальные, которые способны выполнять множество метазадач.

In [6]:
import os
from getpass import getpass
import warnings
warnings.filterwarnings('ignore')

In [65]:
# # Если используете ключ от OpenAI, запустите эту ячейку
# from langchain_openai import ChatOpenAI

# # инициализируем языковую модель
# llm = ChatOpenAI(temperature=0.0)

In [ ]:
from utils import ChatOpenAI
import os 

course_api_key = os.environ['LLM_STEPIK_KEY']

# инициализируем языковую модель
llm = ChatOpenAI(temperature=0.0, course_api_key=course_api_key)

 #  <center id="c2"> 🛠🪚  Инструменты для агентов 🤖

Для использования агентов нам понадобятся 3 компонента:
* LLM
* Инструменты (Tools)
* Сами агенты

Теперь давайте создадим наш первый `Tool` - калькулятор, используя уже знакомую нам `LLMMathChain`:

In [3]:
from langchain.chains import LLMMathChain
from langchain.agents import Tool

llm_math = LLMMathChain.from_llm(llm=llm)

# создаём math tool, задаём имя, описание и функцию
math_tool = Tool(
    name='Калькулятор',
    func=llm_math.run,
    description='Может производить математические расчёты.'
)

# Для передачи в агента, нам нужно поместить инструменты в список
tools = [math_tool]

In [4]:
tools[0].name, tools[0].description

('Калькулятор', 'Может производить математические расчёты.')

Создадим нашего **первого агента**, передав в него инструменты (пока 1)

In [5]:
from langchain.agents import initialize_agent

first_agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

/home/jovyan/.local/miniconda3/lib/python3.10/site-packages/langchain_core/_api/deprecation.py:117: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc. instead.
  warn_deprecated(


In [27]:
first_agent('''У Маши было 3 яблока, Гриша принёс ещё 2 с половиной ящика яблок.
            Сколько всего яблок у ребят, если ящик вмещает 8 яблок?''')



> Entering new AgentExecutor chain...
I need to calculate the total number of apples.
Action: Калькулятор
Action Input: 3 + (2.5 * 8)
Observation: Answer: 23.0
Thought:The final answer is 23.
Observation: Invalid Format: Missing 'Action:' after 'Thought:
Thought:I made a mistake in the format. I need to provide the correct format.
Final Answer: The total number of apples is 23.

> Finished chain.


{'input': 'У Маши было 3 яблока, Гриша принёс ещё 2 с половиной ящика яблок.\n            Сколько всего яблок у ребят, если ящик вмещает 8 яблок?',
 'output': 'The total number of apples is 23.'}

Если мы хотим, чтобы агент мог отвечать на `general knowledge questions` (общие вопросы), можно добавить саму LLM как инструмент `LLM-tool`.

In [36]:
# создаём LLM tool
llm_tool = Tool(
    name='Language Model',
    func=llm.predict,
    description='Отвечает на общие вопросы'
)

# Добавляем его к другим инструментам
tools.append(llm_tool)

In [37]:
first_agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

In [38]:
first_agent("Столица Хорватии?")



> Entering new AgentExecutor chain...
I know that the capital of Croatia is Zagreb, but I can also confirm it using the language model.
Action: Language Model
Action Input: "What is the capital of Croatia?"
Observation: The capital of Croatia is Zagreb.
Thought:I now know the final answer
Final Answer: The capital of Croatia is Zagreb.

> Finished chain.


{'input': 'Столица Хорватии?', 'output': 'The capital of Croatia is Zagreb.'}

### Иногда из-за особенностей токенизации, модель может ошибаться в ответах на достаточно простые вопросы:

In [19]:

llm.invoke("Сколько букв в слове зачёт?")

AIMessage(content='В слове "зачёт" 6 букв.')



Рассмотрим ещё один способ создания инструмента из Python-функции, при помощи декоратора `tool`.

В функцию обязательно необходимо добавлять описание (`docstring`), чтобы агент понимал для чего её нужно применять!

In [21]:
from langchain.agents import tool

@tool
def get_word_length(word: str) -> int:
    """Возвращает длину слова""" # добавляем docstring
    return len(word)


tools = [get_word_length]



Создадим агента с нашим инструментом, и посмотрим как он справится с задачей:

In [23]:
agent = initialize_agent(
    tools=tools,
    llm=llm,
    verbose=True,
)

agent("Сколько букв в слове зачёт?")



> Entering new AgentExecutor chain...
I need to find the length of the word "зачёт".
Action: get_word_length
Action Input: "зачёт"
Observation: 5
Thought:The word "зачёт" has 5 letters.
Final Answer: 5

> Finished chain.


{'input': 'Сколько букв в слове зачёт?', 'output': '5'}

# <center id="c3"> 🐍 Python Agent 😎

In [42]:
from langchain_experimental.tools.python.tool import PythonREPLTool
from langchain.agents import initialize_agent

py_agent = initialize_agent(
    tools=[PythonREPLTool()],
    llm=llm,
    verbose=True,
    handle_parsing_errors=True,
    max_iterations=3
)

In [44]:
names = ["Борис", "Олег", "Ия", "Александр", "Зоя"]

In [45]:
py_agent.run(f'Отсортируй имена в списке по длине в порядке убывания и покажи результат: {names}')



> Entering new AgentExecutor chain...


Python REPL can execute arbitrary code. Use with caution.


I can use the `sorted()` function with a custom key to sort the names by length in descending order.
Action: Python_REPL
Action Input: `names = ['Борис', 'Олег', 'Ия', 'Александр', 'Зоя']`
Observation: 
Thought:I can use the `sorted()` function with a custom key to sort the names by length in descending order.
Action: Python_REPL
Action Input: `sorted(names, key=lambda x: len(x), reverse=True)`
Observation: 
Thought:I now know the final answer
Final Answer: ['Александр', 'Борис', 'Олег', 'Зоя', 'Ия']

> Finished chain.


"['Александр', 'Борис', 'Олег', 'Зоя', 'Ия']"

## Создаём первый мини-сервис


Теперь давайте попробуем реализовать "полезного" агента с использованием нескольких инструментов.
В этом примере мы будем использовать инструменты `Wikipedia`, `DuckDuckGo` и `Arxiv`, чтобы создать простой агент для написания эссе. <br> [Здесь](https://python.langchain.com/docs/integrations/tools/) доступен длинный список инструментов, которые агенты могут использовать для взаимодействия с внешним миром.

In [5]:
from langchain.agents import load_tools
from langchain.tools import DuckDuckGoSearchRun

search = DuckDuckGoSearchRun()

# инструменты, поддерживаемые LangChain "из коробки", можно подгружать функцией load_tools
tools = load_tools(["arxiv", "wikipedia"], llm=llm)

tools.extend([  
    Tool(  
        name="Search",  
        func=search.run,  
        description="useful for when you need to answer questions about current events."  
    ),  
])

essay_agent = initialize_agent(
    tools, 
    llm, 
    verbose=True, 
    handle_parsing_errors=True,
    max_iterations=5)

In [7]:
prompt = "Write a 1000-word essay on topic: {topic}, use the tools to retrieve the necessary information."  
topic = "Style Quentin Tarantino in cinema"  

print(essay_agent.run(prompt.format(topic=topic)))



> Entering new AgentExecutor chain...
I need to gather information about Quentin Tarantino's style in cinema. I can use Wikipedia to get an overview of his career and arxiv to find any scholarly articles or research papers on his style.
Action: Wikipedia
Action Input: Quentin Tarantino
Observation: Page: Quentin Tarantino
Summary: Quentin Jerome Tarantino (; born March 27, 1963) is an American film director, screenwriter, and actor. His films are characterized by stylized violence, extended dialogue including a pervasive use of profanity, and references to popular culture.
Tarantino began his career as an independent filmmaker with the release of the crime film Reservoir Dogs in 1992. His second film, Pulp Fiction (1994), a dark comedy crime thriller, was a major success with critics and audiences winning numerous awards, including the Palme d'Or and the Academy Award for Best Original Screenplay. In 1996, he appeared in From Dusk till Dawn, also writing the screenplay. Tarantino's t



Агент, в целом, справился с задачей, видно, что сначала он сходил в Википедию, затем в Arxiv, потом выдал требуемое эссэ. 
Но в `Final answer` вместо эссэ на 1000 слов агент записал краткий ответ на вопрос, видимо, нужно получше составить промпт (можете потюнить промпт в качестве самостоятельной работы).
    
Тут мог возникнуть справедливый вопрос: А зачем использовать агента, если можно сделать тоже самое, например, с помощью `RouterChain`?
В отличии от цепочек, где порядок действий жёстко прописан, агент получает на вход запрос (промпт), список доступных инструментов и сам решает какие именно инструменты и в каком порядке применить для решения задачи.

### Так же с помощью `load_tools` можно подгрузить такой интересный tool:
```python
tools = load_tools(["human"])
```

В документации к этому tool сказано:
> Human are AGI so they can certainly be used as a tool to help out AI agent when it is confused.

[Здесь](https://python.langchain.com/docs/integrations/tools/human_tools) можно прочитать про этот инструмент подробнее, пока ещё ИИ не может справиться без нас 🫣